In [1]:
from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/storage/Full_currency/Full_currency/Augment_full224*224',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/storage/Full_currency/Full_currency/Augment_full224*224', # same directory as training data
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',subset='validation')

Found 3218 images belonging to 10 classes.
Found 798 images belonging to 10 classes.


In [3]:
def build_model():
    # constructing the model using a pre-trained model on ImageNet
    model = applications.MobileNet(weights="imagenet", include_top=False, input_shape=(224, 224, 3),
                                  pooling='avg')

    # only train the last 2 layers
    for layer in model.layers[:-2]:
        layer.trainable = False

    # output for all 10 classes
    predictions = Dense(10, activation="softmax")(model.output)

    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)
    
    return model_final

model_final = build_model()
# compile the model
model_final.compile(loss="categorical_crossentropy", optimizer= optimizers.adam(lr = 0.001), metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [4]:

# Save the model according to the conditions
checkpoint = ModelCheckpoint("currency_denominator.h5", monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False,
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

In [5]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 16,
    epochs = 5,
    validation_data=validation_generator,
    validation_steps = validation_generator.samples // 16,
    workers=16,
    callbacks=[checkpoint, early])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
201/201 [==============================] - 120s 598ms/step - loss: 0.8352 - acc: 0.7668 - val_loss: 0.7787 - val_acc: 0.7602

Epoch 00001: val_loss improved from inf to 0.77872, saving model to currency_denominator.h5
Epoch 2/5
201/201 [==============================] - 117s 584ms/step - loss: 0.1860 - acc: 0.9726 - val_loss: 0.4401 - val_acc: 0.9207

Epoch 00002: val_loss improved from 0.77872 to 0.44008, saving model to currency_denominator.h5
Epoch 3/5
201/201 [==============================] - 117s 582ms/step - loss: 0.0952 - acc: 0.9894 - val_loss: 0.4134 - val_acc: 0.8990

Epoch 00003: val_loss improved from 0.44008 to 0.41336, saving model to currency_denominator.h5
Epoch 4/5
201/201 [==============================] - 117s 582ms/step - loss: 0.0657 - acc: 0.9944 - val_loss: 0.2748 - val_acc: 0.9591

Epoch 00004: val_loss improved from 0.41336 to 0.27480, saving model to currency_denominator.h5
Epoch 5/5
201/201 [=========

In [6]:
# model save for prediction 
from keras.models import load_model
import h5py

In [7]:
model = load_model('/storage/Full_currency/currency_denominator.h5')

In [8]:
# prediction for image
from keras.preprocessing import image
import numpy as np

In [9]:
# load image convert to numpy array and rescaling it
img = image.load_img('/storage/Full_currency/test7.png', target_size=(224, 224))
x = image.img_to_array(img) / 255 
x = np.expand_dims(x, axis=0)

In [11]:
import matplotlib.pyplot as plt


In [ ]:
plt.imshow(img)

In [ ]:

generator= train_datagen.flow_from_directory("/storage/Full_currency/Full_currency/Augment_224*224", batch_size=16)
label_map = generator.class_indices

In [ ]:
label_map

In [ ]:
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)

In [ ]:
print(list(label_map.keys())[list(label_map.values()).index(y_classes[0])])

In [ ]:
# Save Model

model_final.save('10_class_5_epoch.h6')